In [1]:
import numpy as np
from game_tools import NormalFormGame
from logitdyn import LogitDynamics
from __future__ import division

In [29]:
coo_payoffs = np.array([[4,0], [3,2]])
g_coo = NormalFormGame(coo_payoffs)

### test_simulate_LLN

In [43]:
u = coo_payoffs
beta = 1.0
P = np.zeros((2,2))

I made a probabilistic choice matrix $P$ in a redundant way just in case.

In [44]:
P[0,0] = np.exp(u[0,0] * beta) / (np.exp(u[0,0] * beta) + np.exp(u[1,0] * beta))
P[0,0]

0.73105857863000479

In [45]:
P[1,0] = np.exp(u[1,0] * beta) / (np.exp(u[0,0] * beta) + np.exp(u[1,0] * beta))
P[1,0]

0.2689414213699951

In [46]:
P[0,1] = np.exp(u[0,1] * beta) / (np.exp(u[0,1] * beta) + np.exp(u[1,1] * beta))
P[0,1]

0.11920292202211755

In [47]:
P[1,1] = np.exp(u[1,1] * beta) / (np.exp(u[0,1] * beta) + np.exp(u[1,1] * beta))
P[1,1]

0.88079707797788243

In [48]:
print P

[[ 0.73105858  0.11920292]
 [ 0.26894142  0.88079708]]


$P[i,j]$ represents the probability that a player chooses an action $i$ provided that his opponent takes an action $j$.

In [49]:
Q = np.zeros((4,4))
Q[0, 0] = P[0, 0]
Q[0, 1] = 0.5 * P[1, 0]
Q[0, 2] = 0.5 * P[1, 0]
Q[0, 3] = 0
Q[1, 0] = 0.5 * P[0, 0]
Q[1, 1] = 0.5 * P[0, 1] + 0.5 * P[1, 0]
Q[1, 2] = 0
Q[1, 3] = 0.5 * P[1, 1]
Q[2, 0] = 0.5 * P[0, 0]
Q[2, 1] = 0
Q[2, 2] = 0.5 * P[1, 0] + 0.5 * P[0, 1]
Q[2, 3] = 0.5 * P[1, 1]
Q[3, 0] = 0
Q[3, 1] = 0.5 * P[0, 1]
Q[3, 2] = 0.5 * P[0, 1]
Q[3, 3] = P[1, 1]
print Q

[[ 0.73105858  0.13447071  0.13447071  0.        ]
 [ 0.36552929  0.19407217  0.          0.44039854]
 [ 0.36552929  0.          0.19407217  0.44039854]
 [ 0.          0.05960146  0.05960146  0.88079708]]


$Q$ is the transition probability matrix. The first row and column represent the state $(0,0)$, which means that player 1 takes action 0 and player 2 also takes action 0. The second ones  represent $(0,1)$, the third ones represent $(1,0)$, and the last ones represent $(1,1)$.

In [52]:
from quantecon.mc_tools import MarkovChain

In [53]:
mc = MarkovChain(Q)

In [55]:
mc.stationary_distributions[0]

array([ 0.22451524,  0.08259454,  0.08259454,  0.61029569])

I take 0.61029569 as the criterion for the test.

In [56]:
ld = LogitDynamics(g_coo)

In [130]:
# New one  (using replicate)
n = 1000
seq = ld.replicate(T=100, num_reps=n)
count = 0
for i in range(n):
    if all(seq[i, :] == [1, 1]):
        count += 1
ratio = count / n
ratio

0.605

In [131]:
# Old one
counts = np.zeros(1000)
for i in range(1000):
    seq = ld.simulate(ts_length=100)
    count = 0
    for j in range(100):
        if all(seq[j, :] == [1, 1]):
            count += 1
    counts[i] = count
m = counts.mean() / 100
m

0.59267999999999998